# Doing a Linear regression without Scikit Learn

There's nothing better to understand the gradient descent algorithm than to code it from scratch. It may seem difficult at first glance but don't worry, we will guide you step by step. It is also a fantastic occasion to practice python key concepts like **Classes**!

Don't hesitate to come back to your Machine Learning course on linear regression to refresh your memory. 

Our goal will be to code a simple linear regression such as : 

$f(x) = \beta_1 \times x + \beta_0$

* Import the following libraries: 
  * Numpy 

In [31]:
import numpy as np 

* Define a `Model` class that will take two methods: 
  1. `__init__(self)`, the class builder which will allow you to define an attribute $\beta_0$ (`beta_0` in your code) and an attribute $\beta_1$ (`beta_1` in your code). These attributes represent the coefficients/parameters of the model an we will be initialize them randomly using Numpy (cf: `np.random.randn`).
  2. `__call__(self, x)`, a special method that will turn our class into a callable which will return $\beta_1 \times x + \beta_0$ when called.

In [32]:
class Model() :

    def __init__(self):
        self.beta_1 = np.random.randn(1)
        self.beta_0 = np.random.randn(1)

    def __call__(self, x):
        return self.beta_0 + self.beta_1 * x

* Create an instance of your class `Model`

In [33]:
model = Model()
print(f"beta_0 : {model.beta_0} | beta_1 : {model.beta_1}")

beta_0 : [0.9460767] | beta_1 : [-1.37497872]


* Try doing a first "regression" by running `model(3.0)`. 
NB: If you don't have the same values as this notebook in output, this is normal since you have initialized your values randomly. 

In [34]:

print(f"Using default callable method with x=3.0 : {model(3.0)}")

Using default callable method with x=3.0 : [-3.17885945]


* This value corresponds to a random prediction of your model. But we don't have any data yet. This time, let's use `sklearn` to import data. 
  * Import `sklearn.datasets`
  * Use the `load_diabetes()` function to load the diebetes dataset in an object called `diabetes`.
  * Print the `DESCR` attribute of the diabetes object
  * Save the content of the `data` attribute in an object named `diabetes_data`
  * Save the content of the `target` attribute in an object named `y`

In [35]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()

print(diabetes.DESCR)

diabetes_data = diabetes.data
y = diabetes.target

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

:Number of Instances: 442

:Number of Attributes: First 10 columns are numeric predictive values

:Target: Column 11 is a quantitative measure of disease progression one year after baseline

:Attribute Information:
    - age     age in years
    - sex
    - bmi     body mass index
    - bp      average blood pressure
    - s1      tc, total serum cholesterol
    - s2      ldl, low-density lipoproteins
    - s3      hdl, high-density lipoproteins
    - s4      tch, total cholesterol / HDL
    - s5      ltg, possibly log of serum triglycerides level
    - s6      glu, blood sugar level

Note: Each of these 10 feature variables have bee

* We have too much data in this dataset `diabetes_data`, take only the third column of the dataset and store it in a `diabetes_X` variable.

In [36]:
diabetes_X = diabetes_data[:,2]
diabetes_X[:5]

array([ 0.06169621, -0.05147406,  0.04445121, -0.01159501, -0.03638469])

* Visualize your data using `plotly`.

In [ ]:
import plotly.express as px

px.scatter(x=diabetes_X,y=y)

In [37]:
import plotly.graph_objects as go

fig = go.Figure(
    go.Scatter(
        x=diabetes_X,
        y=y,
        mode='markers'
    )
)

fig.show()


* Now we need to define a cost function. For a linear regression, we could use MSE :

`np.mean((model(input) - y)**2)`

  * Create this function which we'll call `mse` (for mean square error). This function will take two arguments `y_pred` & `y_true`.

In [38]:
def mse(y_pred, y_true):
    return np.mean((y_pred - y_true)**2)

* Test your function by inserting `model(diabetes_X)` & `y` as arguments. 
* Calculate the rmse as well

In [ ]:
print(f"MSE : {mse(model(diabetes_X), y)}")
print(f"RMSE : {np.sqrt(mse(model(diabetes_X), y))}")

MSE : 28793.42837899303
RSME : 169.68626455607134


* Visualize your regression in relation to your points 

In [40]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=diabetes_X,
        y=y,
        mode='markers'             
    )
)

fig.add_trace(
    go.Scatter(
        x=diabetes_X,
        y=model(diabetes_X),
        mode='lines'
    )
)

fig.show()

* We're going to need to compute the gradients for our variable `model.beta_1` and our constant `model.beta_0`. To do this, we're going to need to review our derivative formulas. Since we're not here to do math, we're going to give you these formulas. 
  * `derive_model_beta_1 = 2/len(y_pred)*np.sum((x @ (y_pred - y_true)))`
  * `derive_model_beta_0 = 2/len(y_pred)*(np.sum(y_pred - y_true))`

  * Feel free to read this article if you want to know more about the calculation of the derivative: [Gradient Descent Derivation](https://mccormickml.com/2014/03/04/gradient-descent-derivation/)


  * So using the above formulas, code the first function `derivative_mse_beta_1` that will take the arguments: 
    * `x` --> the values for your variable / `y_pred` --> the values predicted by your model / `y_true` --> the values of the target variable


In [41]:
def derivative_mse_beta_1(x, y_pred, y_true):
    return 2/len(y_pred)*np.sum((x @ (y_pred - y_true)))

* Test you function

In [42]:
derivative_mse_beta_1(diabetes_X, model(diabetes_X), y)

-4.3023087742092025

* So using the above formulas, now code the `derivative_mse_beta_0` function which will take the arguments :
    * `y_pred` --> the values predicted by your model / `y_true` --> the actual values to predict

In [43]:
def derivative_mse_beta_0(y_pred, y_true):
    return 2/len(y_pred)*(np.sum(y_pred - y_true))

* Test you function

In [44]:
derivative_mse_beta_0(model(diabetes_X), y)

-302.3748149326412

* We will try to see if we can minimize our cost function using the two gradients above. To update our variables, we need to subtract their respective gradients. Ex: 
  * `param = param - learning_rate * gradient`

  * Set a `learning_rate` to 0.1
  * Try to apply your formula on `model.beta_1` and `model.beta_0`.

In [45]:
learning_rate = 0.1

model.beta_0 -= learning_rate * derivative_mse_beta_0(model(diabetes_X), y)
model.beta_1 -= learning_rate * derivative_mse_beta_1(diabetes_X, model(diabetes_X), y)

print(f"beta_0 updated : {model.beta_0}")
print(f"beta_1 updated : {model.beta_1}")

beta_0 updated : [31.18355819]
beta_1 updated : [-0.94474784]


We see that the values of the two parameters have changed, let's see how it affected the predictions of the model. 
Visualize the data vs the model's predictions.

In [46]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=diabetes_X,
        y=y,
        mode='markers'             
    )
)

fig.add_trace(
    go.Scatter(
        x=diabetes_X,
        y=model(diabetes_X),
        mode='lines'
    )
)

fig.show()

We notice the predictions got a little closer to our real data
* Recalculate your MSE

In [47]:
print(f"MSE : {mse(model(diabetes_X), y)}")
print(f"RSME : {np.sqrt(mse(model(diabetes_X), y))}")

MSE : 20562.830228189887
RSME : 143.39745544531075


* Our MSE has dropped a lot! This is good news but the process of gradient descent is iterative. So you'll have to do it several times before arriving at accurate predictions. 
  * By making a loop, try to repeat the process from above 10,000 times. 
  * Display every 1000 epochs: mse, model.beta_1 & model.beta_0 

In [49]:
epochs = 1000

for i in range(epochs) :
    
    model.beta_0 -= learning_rate * derivative_mse_beta_0(model(diabetes_X), y)
    model.beta_1 -= learning_rate * derivative_mse_beta_1(diabetes_X, model(diabetes_X), y)

    print(f"beta_0 : {model.beta_0} | beta_1 : {model.beta_1} | MSE : {mse(model(diabetes_X), y)}")

beta_0 : [55.37354338] | beta_1 : [-0.51471164] | MSE : 15294.582881789174
beta_0 : [74.72553154] | beta_1 : [-0.08487002] | MSE : 11922.240651454473
beta_0 : [90.20712206] | beta_1 : [0.3447771] | MSE : 9763.278296106482
beta_0 : [102.59239448] | beta_1 : [0.77422981] | MSE : 8380.879660910941
beta_0 : [112.50061242] | beta_1 : [1.20348819] | MSE : 7495.482406230917
beta_0 : [120.42718677] | beta_1 : [1.63255234] | MSE : 6928.166634156258
beta_0 : [126.76844625] | beta_1 : [2.06142234] | MSE : 6564.423609482436
beta_0 : [131.84145383] | beta_1 : [2.49009829] | MSE : 6330.967741137016
beta_0 : [135.8998599] | beta_1 : [2.91858026] | MSE : 6180.896250292597
beta_0 : [139.14658475] | beta_1 : [3.34686835] | MSE : 6084.191357959083
beta_0 : [141.74396463] | beta_1 : [3.77496265] | MSE : 6021.641685042743
beta_0 : [143.82186854] | beta_1 : [4.20286324] | MSE : 5980.951948384013
beta_0 : [145.48419166] | beta_1 : [4.6305702] | MSE : 5954.253166221677
beta_0 : [146.81405016] | beta_1 : [5.05

* Using `plotly`, view your model and actual values again

In [50]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=diabetes_X,
        y=y,
        mode='markers'             
    )
)

fig.add_trace(
    go.Scatter(
        x=diabetes_X,
        y=model(diabetes_X),
        mode='lines'
    )
)

fig.show()

**We've got a nice regression this time!** 